In [11]:
import os
import re
import json
import pytesseract
from PIL import Image
from docx import Document
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.schema import HumanMessage

In [12]:
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [13]:
folder_path = "./Data"

for filename in os.listdir(folder_path):
    if filename.lower().endswith((".png", ".jpg", ".jpeg")):
        image_path = os.path.join(folder_path, filename)
        image = Image.open(image_path)
        text = pytesseract.image_to_string(image)
        print(f"Text from {filename}:")
        print(text)
        print("-" * 50)
all_texts = []
text = "\n".join(all_texts)


Text from Screenshot 2025-10-14 100949.png:
Development of a "Hand Control" App

''d like to propose the development of a new application that replicates the innovative hand control
functionality seen in Huawei and Honor devices. This "Hand Control" app will allow users to control
‘their smartphones using gestures without touching the screen, enhancing accessibility and user
convenience.

To build an Android application that uses the front-facing camera to recognize specific hand gestures
and translates them into phone commands, providing a true touch-free experience.

2. Key Features to Implement

Scroll: A simple hand wave up or down to scroll through web pages, social media feeds, or lists.
‘Swipe: A left or right palm swipe to browse photos, switch between home screens, or navigate
back/forward.

Screenshot: A fist clench to capture the screen.

- Smart Activation: The system only activates when the user is looking at the screen, preventing
accidental triggers and saving battery.



In [14]:
OUTPUT_DOCX = "project_proposal.docx"

prompt_text = f"""
You are an expert project analyst.
Categorize the following text and infer client requirements.
Output the result in valid JSON ONLY. Each field that is a list must be an actual JSON list of strings.
Use this template exactly:
{{
    "Project Overview": {{
        "Project Title": "",
        "Client Name": "",
        "Project Summary": "",
        "Objectives": []
    }},
    "Business Requirements": {{
        "Goals": [],
        "Target Audience": "",
        "Expected Outcomes": [],
        "Success Metrics": [],
        "Monetization Strategy": ""
    }},
    "Technical Requirements": {{
        "Core Features": [],
        "Core Features & Functionalities": [],
        "Tech Stack": [],
        "Integration Needs": [],
        "Security & Compliance": [],
        "Performance Criteria": []
    }},
    "App Flow": {{
        "App Flow Summary": []
    }},
    "Project Scope": {{
        "Inclusions": [],
        "Exclusions": [],
        "Deliverables": [],
        "Milestones": [],
        "Estimated Timeline & Pricing": ""
    }},
    "Timeline & Resources": {{
        "Estimated Duration": "",
        "Team Roles": [],
        "Dependencies": []
    }},
    "Budget & Costing": {{
        "Estimated Budget": "",
        "Cost Breakdown": {{}}
    }},
    "Risk Assessment": {{
        "Potential Risks": [],
        "Mitigation Strategies": []
    }},
    "Other Notes": ""
}}
Text: {json.dumps(text)}
"""

messages = [HumanMessage(content=prompt_text)]

# -------------------------------
# STEP 3: Send to Groq LLM
# -------------------------------
import os

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY environment variable not set!")

client = ChatGroq(
    groq_api_key=GROQ_API_KEY,
    model="openai/gpt-oss-120b",
    temperature=1.5
)

response = client(messages)
llm_output = response.content.strip()

try:
    structured_output = json.loads(llm_output)
except json.JSONDecodeError:
    # fallback: extract first JSON block from text
    json_match = re.search(r"\{.*\}", llm_output, re.DOTALL)
    if json_match:
        structured_output = json.loads(json_match.group(0))
    else:
        raise ValueError("Could not parse JSON from LLM output!")

doc = Document()
doc.add_heading("Project Proposal", 0)

def add_section(section_name, content):
    """Add a section to the Word document"""
    doc.add_heading(section_name, level=1)
    for key, value in content.items():
        # Special handling for Budget & Costing
        if section_name == "Budget & Costing":
            if key == "Estimated Budget":
                doc.add_heading("Estimated Budget", level=2)
                doc.add_paragraph(str(value))
            elif key == "Cost Breakdown":
                doc.add_heading("Cost Breakdown", level=2)
                if isinstance(value, dict) and value:
                    table = doc.add_table(rows=1, cols=2)
                    table.style = "Light Grid"
                    hdr_cells = table.rows[0].cells
                    hdr_cells[0].text = "Item"
                    hdr_cells[1].text = "Amount"
                    for item_name, amount in value.items():
                        row_cells = table.add_row().cells
                        row_cells[0].text = str(item_name)
                        row_cells[1].text = str(amount)
                else:
                    doc.add_paragraph("No cost breakdown provided.")
            continue

        doc.add_heading(key, level=2)
        if isinstance(value, list):
            for item in value:
                doc.add_paragraph(item, style="List Bullet")
        elif isinstance(value, dict):
            for sub_key, sub_value in value.items():
                doc.add_paragraph(f"{sub_key}: {sub_value}", style="List Bullet")
        else:
            doc.add_paragraph(str(value))

# Sections to include
sections = [
    "Project Overview",
    "Business Requirements",
    "Technical Requirements",
    "App Flow",
    "Project Scope",
    "Timeline & Resources",
    "Budget & Costing",
    "Risk Assessment"
]

for section in sections:
    content = structured_output.get(section, {})
    if content:
        add_section(section, content)

# Other Notes
other_notes = structured_output.get("Other Notes", "")
if other_notes:
    doc.add_heading("Other Notes", level=1)
    doc.add_paragraph(other_notes)

# Save Document
doc.save(OUTPUT_DOCX)
print(f"✅ Project proposal created successfully: {OUTPUT_DOCX}")


C:\Users\Bd Calling\AppData\Local\Temp\ipykernel_2292\1007612490.py:75: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = client(messages)


✅ Project proposal created successfully: project_proposal.docx
